In [ ]:
import pandas as pd
from Bio import SeqIO

# download from String database
file_path = '/data4/lyk/manything/9606.protein.sequences.v12.0.fa'


data = []


for record in SeqIO.parse(file_path, "fasta"):
    data.append({"id": record.id, "sequence": str(record.seq)})


df = pd.DataFrame(data)
print(df.shape)
df.head()

(19699, 2)


,id,sequence
0,9606.ENSP00000000233,MGLTVSALFSRIFGKKQMRILMVGLDAAGKTTILYKLKLGEIVTTI...
1,9606.ENSP00000000412,MFPFYSCWRTGLLLLLLAVAVRESWQTEEKTCDLVGEKGKESEKEL...
2,9606.ENSP00000001008,MTAEEMKATESGAQSAPLPMEGVDISPKQDEGVLKVIKREGTGTEM...
3,9606.ENSP00000001146,MLFEGLDLVSALATLAACLVSVTLLLAVSQQLWQLRWAATRDKSCK...
4,9606.ENSP00000002125,MSVLLRSGLGPLCAVARAAIPFIWRGKYFSSGNEPAENPVTPMLRH...


In [ ]:
import pandas as pd
# download from String database
pros = pd.read_csv('/data4/lyk/manything/9606.protein.info.v12.0.txt', sep='\t')
pros.head()

,#string_protein_id,preferred_name,protein_size,annotation
0,9606.ENSP00000000233,ARF5,180,ADP-ribosylation factor 5; GTP-binding protein...
1,9606.ENSP00000000412,M6PR,277,Cation-dependent mannose-6-phosphate receptor;...
2,9606.ENSP00000001008,FKBP4,459,"Peptidyl-prolyl cis-trans isomerase FKBP4, N-t..."
3,9606.ENSP00000001146,CYP26B1,512,Cytochrome P450 26B1; Involved in the metaboli...
4,9606.ENSP00000002125,NDUFAF7,441,"Protein arginine methyltransferase NDUFAF7, mi..."


In [ ]:
# download from String database
ppis = pd.read_csv('/data4/lyk/manything/9606.protein.links.v12.0.txt', sep=' ')
df_merged = ppis.merge(pros[['#string_protein_id', 'preferred_name']], 
                                   left_on='protein1', 
                                   right_on='#string_protein_id', 
                                   how='left')


df_merged['protein1'] = df_merged['preferred_name']


df_merged.drop(columns=['#string_protein_id', 'preferred_name'], inplace=True)


df_merged = df_merged.merge(pros[['#string_protein_id', 'preferred_name']], 
                                   left_on='protein2', 
                                   right_on='#string_protein_id', 
                                   how='left')


df_merged['protein2'] = df_merged['preferred_name']


df_merged.drop(columns=['#string_protein_id', 'preferred_name'], inplace=True)

ppis = df_merged
ppis.head()

,protein1,protein2,combined_score
0,ARF5,RALGPS2,173
1,ARF5,FHDC1,154
2,ARF5,ATP6V1E1,151
3,ARF5,CYTH2,471
4,ARF5,PSD3,201


In [ ]:
import sys

def display_progress(current, total):
    bar_length = 40  
    progress = current / total
    arrow = '█' * int(round(progress * bar_length) - 1)
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write(f'\rProgress: |{arrow}{spaces}| {current}/{total} ({progress:.2%})')
    sys.stdout.flush()

In [5]:
df_merged.head()

,protein1,protein2,combined_score
0,ARF5,RALGPS2,173
1,ARF5,FHDC1,154
2,ARF5,ATP6V1E1,151
3,ARF5,CYTH2,471
4,ARF5,PSD3,201


In [ ]:
import networkx as nx
# G = nx.read_gml('graph_ppi.gml')
G = nx.read_gexf('../mippi_variation/ppi_network.gexf')

G = nx.Graph()


for protein in pros['preferred_name']:
    G.add_node(protein)


num_now = 0
for _, row in df_merged.iterrows():
    G.add_edge(row['protein1'], row['protein2'], weight=row['combined_score'])
    if num_now % 1000 == 0:
        display_progress(num_now, len(df_merged))
    num_now = num_now + 1

source = 'ARF5'
target = 'FHDC1'
shortest_path = nx.shortest_path(G, source=source, target=target, weight='weight')
shortest_path_length = nx.shortest_path_length(G, source=source, target=target, weight='weight')


print(f"最短路径从 {source} 到 {target}: {shortest_path}")
print(f"最短路径长度: {shortest_path_length}")

In [9]:
def get_min(source_node, target_nodes, G):
    shortest_distances = {}
    for target in target_nodes:
        if target in G:
            distance = nx.shortest_path_length(G, source=source_node, target=target)
            shortest_distances[target] = distance

    if shortest_distances:
        min_distance = min(shortest_distances.values())
        # closest_nodes = [node for node, distance in shortest_distances.items() if distance == min_distance]
    else:
        min_distance = None
    return min_distance

In [ ]:
import pandas as pd

# download from https://static-content.springer.com/esm/art%3A10.1038%2Fs41588-018-0130-z/MediaObjects/41588_2018_130_MOESM6_ESM.xlsx
file_path = 'protein_net/41588_2018_130_MOESM6_ESM.xlsx'
sheets = pd.read_excel(file_path, sheet_name=None, header=None)


dataframes = {sheet_name: df for sheet_name, df in sheets.items()}
importants = []

for name, df in dataframes.items():
    print(f"name: {name}")
    # print(len(set(df[0].tolist()) & set(pros['preferred_name'].tolist())), len(df[0].tolist()))  
    print(len([node for node in df[0].tolist() if node in G.nodes]))
    importants.append([node for node in df[0].tolist() if node in G.nodes])


表格名称: Sheet1
73
表格名称: Sheet2
51


In [4]:
G.nodes

NodeView(('MAP2K4', 'FLNC', 'GATA2', 'PML', 'RPA2', 'STAT3', 'ARF1', 'GGA3', 'ARF3', 'ARFIP2', 'ARFIP1', 'APP', 'APPBP2', 'APLP1', 'DAB1', 'CITED2', 'TFAP2A', 'EP300', 'APOB', 'MTTP', 'ARRB2', 'RALGDS', 'CSF1R', 'GRB2', 'PRRC2A', 'SLC4A1', 'SLC4A1AP', 'BCL3', 'BARD1', 'ADRB1', 'GIPC1', 'BRCA1', 'ATF1', 'MSH2', 'MSH6', 'ARVCF', 'CDH15', 'PSEN2', 'CAPN1', 'CAPN3', 'TTN', 'ITPR1', 'CA8', 'MAGI1', 'CTNNB1', 'RB1', 'CUX1', 'TSG101', 'CDKN1A', 'MCM5', 'MCM2', 'CDC7', 'ORC2', 'DBF4', 'AKAP8', 'RPA1', 'RPA3', 'ORC5', 'MCM3', 'YWHAZ', 'CDC25A', 'DOCK8', 'CDC42', 'TRIP10', 'CDKN3', 'CDK2', 'RBBP8', 'RBL1', 'CLTC', 'HGS', 'IGFBP3', 'COL1A1', 'TXN', 'ATF2', 'SMAD3', 'SMAD4', 'CCND3', 'EIF3K', 'RALBP1', 'CCNB1', 'CSRP1', 'CSK', 'IGF1R', 'DCN', 'EGFR', 'SIM2', 'ARNT', 'SIM1', 'NPAS4', 'HBEGF', 'ZBTB16', 'PCNA', 'GADD45A', 'LSM3', 'SNRPE', 'LSM4', 'LSM5', 'EGR1', 'CREBBP', 'NUMA1', 'EPB41', 'PLCG1', 'EEF1A1', 'KTN1', 'EEF1D', 'KDR', 'ANXA5', 'PPARGC1B', 'ESR1', 'SMARCA4', 'DDX54', 'DNTTIP2', 'FGFR1',

In [ ]:
def calculate_distances(G, importants):
    """ Calculate the shortest distance from each important node to all other nodes, and obtain the minimum value """

    all_distances_list = []


    for important_list in importants:
        # if important in G:
        all_distances = {}
        for important in important_list:
            distances = nx.single_source_shortest_path_length(G, important)
            all_distances[important] = distances
        all_distances_list.append(all_distances)
    print('done')

    min_distances = {}
    # return all_distances

    for node in G.nodes:
        # if node not in importants:
        min_distance = float('inf')
        for important in importants:
            if important in all_distances and node in all_distances[important]:
                min_distance = min(min_distance, all_distances[important][node])
        min_distances[node] = min_distance if min_distance != float('inf') else None
        # print_progress_bar(len(min_distances), leeee)
    return min_distances


# ttt = calculate_distances(G, importants)
# ttt
# min_distances = calculate_distances(G, importants)

In [ ]:
import sys
import time

def print_progress_bar(iteration, total, prefix='', suffix='', decimals=1, length=50, fill='█', print_end="\r"):

    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filled_length = int(length * iteration // total)
    bar = fill * filled_length + '-' * (length - filled_length)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end=print_end)
    # 当完成时打印新行
    if iteration == total: 
        print()

In [ ]:
import json
def calculate_distances(G, importants):

    all_distances = {}

    for important in importants:
        if important in G:
            distances = nx.single_source_shortest_path_length(G, important)
            all_distances[important] = distances
        print_progress_bar(len(all_distances), len(importants))

    min_distances = {}


    for node in G.nodes:
        if node not in importants:
            min_distance = float('inf')
            for important in importants:
                if important in all_distances and node in all_distances[important]:
                    min_distance = min(min_distance, all_distances[important][node])
            min_distances[node] = min_distance if min_distance != float('inf') else None
        else:
            min_distances[node] = 0
    return min_distances
dis = []
for i in range(len(importants)):
    min_distances = calculate_distances(G, importants[i])
    dis.append(min_distances)
    with open('distance_CDH.json', 'w') as f:
        json.dump(dis, f)




 |██████████████████████████████████████████████████| 100.0% 
 |██████████████████████████████████████████████████| 100.0% 


In [28]:
with open('manything/distance.json', 'w') as f:
    json.dump(dis, f)

In [10]:
distances = []
for i in range(len(importants)):
    distance = {}
    leeee = len(list(G.nodes))
    for node in list(G.nodes):
        distance[node] = get_min(node, importants[i], G)
        print_progress_bar(len(distance), leeee)
    distances.append(distance)
    print(i)

|██--------------------------------------| 5.44% Complete

KeyboardInterrupt: 

In [9]:
import sys
def print_progress_bar(iteration, total, length=40):
    percent = (iteration / total) * 100
    filled_length = int(length * iteration // total)
    bar = '█' * filled_length + '-' * (length - filled_length)
    sys.stdout.write(f'\r|{bar}| {percent:.2f}% Complete')
    sys.stdout.flush()